#**Практическая работа 7**

##Теоретическая часть

###**Редукция**

Редукция это операция, которая позволяет получить одно итоговое значение из набора элементов массива. Чаще всего это сумма, минимум или максимум. Такая операция часто используется при работе с большими объёмами данных, когда последовательные вычисления на CPU становятся медленными.

На GPU редукция выполняется параллельно с использованием потоков и блоков. Каждый блок обрабатывает свою часть данных и вычисляет частичную сумму. Затем эти частичные результаты объединяются до получения одного итогового значения. Для ускорения вычислений часто используется разделяемая память, так как доступ к ней значительно быстрее, чем к глобальной памяти. Редукция является базовой операцией и применяется во многих задачах, таких как анализ данных и машинное обучение.

###**Сканирование (префиксная сумма)**

Сканирование, или префиксная сумма, это операция, при которой для каждого элемента массива вычисляется сумма всех предыдущих элементов, включая текущий. Например, для массива [1, 2, 3, 4] результатом будет массив [1, 3, 6, 10].

В отличие от редукции, сканирование возвращает массив той же длины, что и входной. Это делает его более сложным для параллельной реализации. На GPU сканирование обычно выполняется в несколько этапов. Сначала вычисляется префиксная сумма внутри каждого блока, затем обрабатываются суммы блоков и добавляются соответствующие смещения. Сканирование широко используется в алгоритмах сортировки, построении гистограмм и обработке изображений.

###**Типы памяти в CUDA**

Производительность программ на GPU во многом зависит от того, какие типы памяти используются. Глобальная память является основной памятью GPU и доступна всем потокам, но она работает медленнее из-за большой задержки доступа. Разделяемая память доступна только потокам внутри одного блока, но при этом она намного быстрее и часто используется для оптимизации параллельных алгоритмов.

Локальная память используется для хранения данных, относящихся к отдельному потоку. Обычно она применяется для временных переменных и промежуточных вычислений. Правильное использование разных типов памяти позволяет значительно повысить скорость выполнения программ на CUDA.

##Практическая часть

##Задание 1

In [1]:
%%writefile reduction.cu

#include <cuda_runtime.h>          // Подключение CUDA Runtime API
#include <iostream>                // Для вывода в консоль
#include <vector>                  // Для использования std::vector
#include <numeric>                 // Для std::accumulate
#include <cmath>                   // Для std::abs

// Макрос для проверки корректности выполнения CUDA-вызовов
#define CHECK_CUDA(call)                                   \
{                                                          \
    cudaError_t err = call;                                 \
    if (err != cudaSuccess) {                              \
        std::cerr << "CUDA error: "                         \
                  << cudaGetErrorString(err)                \
                  << " at line " << __LINE__ << std::endl;  \
        exit(EXIT_FAILURE);                                 \
    }                                                      \
}

// CUDA-ядро для суммирования элементов массива методом редукции
__global__
void reduceSumKernel(const float* d_in, float* d_out, int n)
{
    extern __shared__ float sdata[];        // Объявление разделяемой памяти для частичных сумм

    unsigned int tid = threadIdx.x;         // Индекс потока внутри блока

    unsigned int idx = blockIdx.x           // Индекс блока
                       * blockDim.x * 2     // Учитываем обработку двух элементов на поток
                       + tid;               // Добавляем локальный индекс потока

    float sum = 0.0f;                       // Переменная для хранения частичной суммы

    if (idx < n)                            // Проверка выхода за границы массива
        sum += d_in[idx];                  // Добавление первого элемента

    if (idx + blockDim.x < n)              // Проверка второго элемента
        sum += d_in[idx + blockDim.x];     // Добавление второго элемента

    sdata[tid] = sum;                      // Запись частичной суммы в shared memory
    __syncthreads();                       // Синхронизация потоков внутри блока

    for (unsigned int stride = blockDim.x / 2; // Начальный шаг редукции
         stride > 0;                           // Пока шаг больше нуля
         stride >>= 1)                         // Делим шаг на два
    {
        if (tid < stride)                     // Активны только первые потоки
            sdata[tid] += sdata[tid + stride]; // Суммируем элементы
        __syncthreads();                      // Синхронизация после каждой итерации
    }

    if (tid == 0)                             // Первый поток блока
        d_out[blockIdx.x] = sdata[0];         // Записывает результат блока
}

// Функция на стороне CPU для многошаговой редукции
float gpuReduceSum(const std::vector<float>& h_input)
{
    int n = h_input.size();                   // Размер входного массива

    float* d_in;                              // Указатель на входной массив на GPU
    float* d_out;                             // Указатель на выходной массив на GPU

    CHECK_CUDA(cudaMalloc(&d_in, n * sizeof(float))); // Выделение памяти на GPU
    CHECK_CUDA(cudaMemcpy(d_in, h_input.data(),       // Копирование данных
                           n * sizeof(float),
                           cudaMemcpyHostToDevice));

    const int threads = 256;                  // Количество потоков в блоке

    while (n > 1)                             // Пока не останется один элемент
    {
        int blocks = (n + threads * 2 - 1)    // Вычисление количества блоков
                     / (threads * 2);

        CHECK_CUDA(cudaMalloc(&d_out, blocks * sizeof(float))); // Выделение памяти под результат

        reduceSumKernel<<<blocks, threads,    // Запуск CUDA-ядра
                           threads * sizeof(float)>>>(d_in, d_out, n);

        CHECK_CUDA(cudaDeviceSynchronize());  // Ожидание завершения ядра

        cudaFree(d_in);                       // Освобождение старого входного массива

        d_in = d_out;                         // Результат становится новым входом
        n = blocks;                           // Обновляем размер массива
    }

    float result;                             // Переменная для хранения результата

    CHECK_CUDA(cudaMemcpy(&result, d_in,      // Копирование результата на CPU
                           sizeof(float),
                           cudaMemcpyDeviceToHost));

    cudaFree(d_in);                           // Освобождение памяти GPU

    return result;                            // Возвращаем итоговую сумму
}

int main()
{
    const int N = 1'000'000;                  // Размер тестового массива

    std::vector<float> data(N);               // Создание массива на CPU

    for (int i = 0; i < N; i++)               // Заполнение массива
        data[i] = static_cast<float>(rand()) / RAND_MAX;

    double cpu_sum = std::accumulate(         // Суммирование на CPU
        data.begin(), data.end(), 0.0);

    float gpu_sum = gpuReduceSum(data);        // Суммирование на GPU

    std::cout << "CPU sum: " << cpu_sum << std::endl; // Вывод CPU результата
    std::cout << "GPU sum: " << gpu_sum << std::endl; // Вывод GPU результата

    std::cout << "Absolute error: "            // Вывод абсолютной ошибки
              << std::abs(cpu_sum - gpu_sum)
              << std::endl;

    return 0;                                 // Завершение программы
}

Writing reduction.cu


In [5]:
!nvcc -arch=sm_75 reduction.cu -o reduction
!./reduction

CPU sum: 500007
GPU sum: 500007
Absolute error: 0.0151951


##**Вывод**

В ходе выполнения задания была реализована операция редукции для суммирования элементов массива с использованием технологии CUDA. Для оптимизации доступа к данным применялась разделяемая память, что позволило сократить количество обращений к глобальной памяти и повысить эффективность вычислений на GPU. Реализация основывалась на многошаговой редукции, при которой на каждом этапе уменьшалось количество обрабатываемых элементов до получения итогового значения.

Корректность работы алгоритма была проверена путём сравнения результатов, полученных на GPU и CPU. Полученные значения сумм совпадают с точностью до погрешности вычислений с плавающей точкой. Абсолютная ошибка составила 0.015 при сумме порядка 5×10⁵, что соответствует крайне малой относительной погрешности и является допустимым для вычислений в формате float.

Наблюдаемая погрешность обусловлена различием порядка операций сложения на CPU и GPU, а также ограниченной точностью представления чисел с плавающей точкой. Таким образом, реализованное CUDA-ядро корректно выполняет редукцию массива и демонстрирует ожидаемое поведение параллельных вычислений на графическом процессоре.

##Задание 2

In [8]:
%%writefile prefix_scan.cu

#include <cuda_runtime.h>                 // CUDA Runtime API
#include <iostream>                       // Вывод в консоль
#include <vector>                         // std::vector
#include <cmath>                          // std::abs, std::max

#define CHECK_CUDA(call)                                   \
{                                                          \
    cudaError_t err = call;                                \
    if (err != cudaSuccess) {                              \
        std::cerr << "CUDA error: "                        \
                  << cudaGetErrorString(err)               \
                  << " at line " << __LINE__ << std::endl; \
        exit(EXIT_FAILURE);                                \
    }                                                      \
}

__global__
void blockInclusiveScanAndSumsKernel(const float* d_in, float* d_out, float* d_block_sums, int n)
{
    extern __shared__ float sdata[];       // Shared memory для данных блока

    unsigned int tid = threadIdx.x;        // Индекс потока в блоке
    unsigned int bdim = blockDim.x;        // Размер блока
    unsigned int bid  = blockIdx.x;        // Индекс блока
    unsigned int idx  = bid * bdim + tid;  // Глобальный индекс элемента

    float x = 0.0f;                        // Локальная переменная для элемента
    if (idx < n)                           // Проверка границы массива
        x = d_in[idx];                     // Читаем элемент из global memory

    sdata[tid] = x;                        // Кладём элемент в shared memory
    __syncthreads();                       // Синхронизация всех потоков блока

    for (unsigned int offset = 1;          // Начинаем с offset=1
         offset < bdim;                    // Пока offset меньше размера блока
         offset <<= 1)                     // Удваиваем offset
    {
        float temp = 0.0f;                 // Временная переменная
        if (tid >= offset)                 // Потоки, которые могут читать назад
            temp = sdata[tid - offset];    // Берут значение на offset левее
        __syncthreads();                   // Синхронизация перед обновлением
        sdata[tid] += temp;                // Обновляем inclusive scan в shared
        __syncthreads();                   // Синхронизация после обновления
    }

    if (idx < n)                           // Проверка границы массива
        d_out[idx] = sdata[tid];           // Пишем префикс для элемента

    if (tid == bdim - 1)                   // Последний поток блока
        d_block_sums[bid] = sdata[tid];    // Записывает сумму всего блока
}

__global__
void addBlockOffsetsKernel(float* d_out, const float* d_block_offsets, int n)
{
    unsigned int tid = threadIdx.x;        // Индекс потока в блоке
    unsigned int bdim = blockDim.x;        // Размер блока
    unsigned int bid  = blockIdx.x;        // Индекс блока
    unsigned int idx  = bid * bdim + tid;  // Глобальный индекс

    if (idx < n)                           // Проверка границы массива
        d_out[idx] += d_block_offsets[bid];// Добавляем оффсет блока ко всем элементам блока
}

int main()
{
    const int N = 1024;                    // Размер тестового массива
    std::vector<float> h_in(N);            // Входной массив на CPU
    std::vector<float> h_out(N);           // Выходной массив на CPU
    std::vector<float> h_cpu(N);           // CPU-эталон

    for (int i = 0; i < N; i++)            // Заполняем тестовый массив
        h_in[i] = static_cast<float>(rand()) / RAND_MAX;

    h_cpu[0] = h_in[0];                    // Инициализация CPU-префикса
    for (int i = 1; i < N; i++)            // Последовательный prefix sum на CPU
        h_cpu[i] = h_cpu[i - 1] + h_in[i];

    float* d_in = nullptr;                 // Указатель на вход на GPU
    float* d_out = nullptr;                // Указатель на выход на GPU

    CHECK_CUDA(cudaMalloc(&d_in, N * sizeof(float)));   // Выделяем память под вход
    CHECK_CUDA(cudaMalloc(&d_out, N * sizeof(float)));  // Выделяем память под выход

    CHECK_CUDA(cudaMemcpy(d_in, h_in.data(), N * sizeof(float), cudaMemcpyHostToDevice)); // Копируем вход на GPU

    const int threads = 256;               // Потоков в блоке
    int blocks = (N + threads - 1) / threads; // Количество блоков

    float* d_block_sums = nullptr;         // Суммы блоков на GPU
    CHECK_CUDA(cudaMalloc(&d_block_sums, blocks * sizeof(float))); // Память под суммы блоков

    blockInclusiveScanAndSumsKernel<<<blocks, threads, threads * sizeof(float)>>>(d_in, d_out, d_block_sums, N); // Scan + суммы блоков
    CHECK_CUDA(cudaDeviceSynchronize());   // Ждём завершения

    std::vector<float> h_block_sums(blocks); // Суммы блоков на CPU
    CHECK_CUDA(cudaMemcpy(h_block_sums.data(), d_block_sums, blocks * sizeof(float), cudaMemcpyDeviceToHost)); // Копируем суммы блоков

    std::vector<float> h_block_offsets(blocks); // Оффсеты блоков на CPU
    float running = 0.0f;                  // Накопленная сумма предыдущих блоков
    for (int b = 0; b < blocks; b++) {     // Идём по блокам
        h_block_offsets[b] = running;      // Оффсет блока = сумма всех предыдущих блоков
        running += h_block_sums[b];        // Обновляем накопленную сумму
    }

    float* d_block_offsets = nullptr;      // Оффсеты блоков на GPU
    CHECK_CUDA(cudaMalloc(&d_block_offsets, blocks * sizeof(float))); // Память под оффсеты
    CHECK_CUDA(cudaMemcpy(d_block_offsets, h_block_offsets.data(), blocks * sizeof(float), cudaMemcpyHostToDevice)); // Копируем оффсеты

    addBlockOffsetsKernel<<<blocks, threads>>>(d_out, d_block_offsets, N); // Добавляем оффсеты к каждому блоку
    CHECK_CUDA(cudaDeviceSynchronize());   // Ждём завершения

    CHECK_CUDA(cudaMemcpy(h_out.data(), d_out, N * sizeof(float), cudaMemcpyDeviceToHost)); // Копируем итог на CPU

    float max_error = 0.0f;                // Максимальная ошибка
    for (int i = 0; i < N; i++)            // Сравниваем CPU и GPU
        max_error = std::max(max_error, std::abs(h_cpu[i] - h_out[i]));

    std::cout << "CPU prefix (first 10): "; // Печать первых 10 CPU
    for (int i = 0; i < 10; i++)
        std::cout << h_cpu[i] << " ";
    std::cout << std::endl;

    std::cout << "GPU prefix (first 10): "; // Печать первых 10 GPU
    for (int i = 0; i < 10; i++)
        std::cout << h_out[i] << " ";
    std::cout << std::endl;

    std::cout << "Max absolute error: " << max_error << std::endl; // Печать ошибки

    cudaFree(d_in);                        // Освобождаем вход
    cudaFree(d_out);                       // Освобождаем выход
    cudaFree(d_block_sums);                // Освобождаем суммы блоков
    cudaFree(d_block_offsets);             // Освобождаем оффсеты блоков

    return 0;                              // Завершение
}

Overwriting prefix_scan.cu


In [9]:
!nvcc -arch=sm_75 prefix_scan.cu -o prefix_scan
!./prefix_scan

CPU prefix (first 10): 0.840188 1.23457 2.01767 2.81611 3.72776 3.92531 4.26053 5.02876 5.30654 5.86051 
GPU prefix (first 10): 0.840188 1.23457 2.01767 2.81611 3.72776 3.92531 4.26053 5.02876 5.30654 5.86051 
Max absolute error: 0.00012207


##**Вывод**

В рамках данного задания была реализована операция префиксной суммы (inclusive scan) с использованием технологии CUDA. Для ускорения вычислений применялась разделяемая память, что позволило эффективно выполнять суммирование элементов внутри каждого блока и сократить количество обращений к глобальной памяти. Алгоритм был реализован с учётом блочной структуры GPU и включал этап вычисления сумм блоков и добавления соответствующих оффсетов, что обеспечило корректное вычисление префиксной суммы для всего массива.

Корректность реализации была проверена путём сравнения результатов, полученных на GPU и CPU. Первые элементы префиксной суммы полностью совпали, а максимальная абсолютная погрешность составила 0.00012207. Данная погрешность обусловлена особенностями представления чисел с плавающей точкой в формате float32 и различием порядка выполнения операций сложения при параллельных вычислениях на GPU.

Таким образом, реализованная CUDA-реализация префиксной суммы корректно выполняет поставленную задачу и демонстрирует ожидаемое поведение параллельного алгоритма с использованием разделяемой памяти и многошаговой схемы вычислений.

##Задание 3

In [11]:
%%writefile benchmark.cu
#include <cuda_runtime.h>                 // Подключение CUDA Runtime API
#include <iostream>                       // Потоки ввода-вывода
#include <vector>                         // Контейнер std::vector
#include <chrono>                         // Измерение времени на CPU
#include <cmath>                          // Математические функции
#include <iomanip>                        // Форматированный вывод

using namespace std;                      // Использование пространства имён std

// Макрос для проверки ошибок CUDA
#define CHECK_CUDA(call) do {                                     \
    cudaError_t err = call;                                       \
    if (err != cudaSuccess) {                                     \
        cerr << "CUDA error: "                                    \
             << cudaGetErrorString(err)                           \
             << " at line " << __LINE__ << endl;                  \
        exit(EXIT_FAILURE);                                       \
    }                                                             \
} while(0)

// GPU-ядро: редукция с использованием атомарных операций
__global__
void reduce_atomic_kernel(const float* d_in, float* d_out, int n)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс элемента
    if (i < n)                                     // Проверка выхода за границы
        atomicAdd(d_out, d_in[i]);                 // Атомарное добавление в global memory
}

// GPU-ядро: оптимизированная редукция с использованием shared memory
__global__
void reduce_shared_kernel(const float* d_in, float* d_out, int n)
{
    extern __shared__ float sdata[];                // Разделяемая память блока

    unsigned int tid = threadIdx.x;                 // Индекс потока в блоке
    unsigned int idx = blockIdx.x * blockDim.x * 2 + tid; // Глобальный индекс (2 элемента на поток)

    float sum = 0.0f;                               // Локальная переменная суммы

    if (idx < n)                                   // Проверка первого элемента
        sum += d_in[idx];                          // Чтение из global memory
    if (idx + blockDim.x < n)                      // Проверка второго элемента
        sum += d_in[idx + blockDim.x];             // Чтение второго элемента

    sdata[tid] = sum;                              // Запись частичной суммы в shared memory
    __syncthreads();                               // Синхронизация потоков блока

    for (unsigned int stride = blockDim.x / 2;     // Начальный шаг редукции
         stride > 0;                               // Пока шаг больше нуля
         stride >>= 1)                             // Деление шага пополам
    {
        if (tid < stride)                          // Активные потоки
            sdata[tid] += sdata[tid + stride];     // Суммирование элементов
        __syncthreads();                           // Синхронизация после шага
    }

    if (tid == 0)                                  // Первый поток блока
        d_out[blockIdx.x] = sdata[0];              // Запись суммы блока
}

// Хост-функция: многошаговая редукция на GPU
float gpu_reduce_shared(const float* d_in, int n)
{
    const int threads = 256;                        // Количество потоков в блоке
    int cur_n = n;                                 // Текущий размер массива

    float* d_prev = nullptr;                       // Указатель на текущий вход
    float* d_curr = nullptr;                       // Указатель на текущий выход

    CHECK_CUDA(cudaMalloc(&d_prev, n * sizeof(float))); // Выделение памяти
    CHECK_CUDA(cudaMemcpy(d_prev, d_in, n * sizeof(float), cudaMemcpyDeviceToDevice)); // Копирование

    while (cur_n > 1)                              // Пока не останется один элемент
    {
        int blocks = (cur_n + threads * 2 - 1) / (threads * 2); // Количество блоков
        CHECK_CUDA(cudaMalloc(&d_curr, blocks * sizeof(float))); // Память под результат

        reduce_shared_kernel<<<blocks, threads, threads * sizeof(float)>>>(d_prev, d_curr, cur_n); // Запуск ядра
        CHECK_CUDA(cudaDeviceSynchronize());        // Ожидание завершения

        cudaFree(d_prev);                           // Освобождение старого массива
        d_prev = d_curr;                            // Новый вход
        d_curr = nullptr;                           // Обнуление указателя
        cur_n = blocks;                             // Обновление размера
    }

    float result = 0.0f;                            // Переменная результата
    CHECK_CUDA(cudaMemcpy(&result, d_prev, sizeof(float), cudaMemcpyDeviceToHost)); // Копирование на CPU
    cudaFree(d_prev);                               // Освобождение памяти

    return result;                                 // Возврат результата
}

// CPU-реализация редукции
float cpu_reduce(const vector<float>& a)
{
    double sum = 0.0;                               // Используем double для точности
    for (float x : a)                               // Последовательный проход
        sum += x;                                  // Суммирование
    return static_cast<float>(sum);                // Возврат результата
}

// Основная функция бенчмарка
int main()
{
    cout << fixed << setprecision(3);               // Форматированный вывод чисел

    vector<int> sizes = {1024, 4096, 16384, 65536, 262144, 1048576}; // Размеры массивов


    cout << setw(10) << "N"
         << setw(15) << "CPU (мс)"
         << setw(20) << "GPU atomic (мс)"
         << setw(20) << "GPU shared (мс)"
         << endl;

    for (int N : sizes)                              // Перебор размеров массивов
    {
        vector<float> h(N);                          // Массив на CPU
        for (int i = 0; i < N; i++)                  // Заполнение массива
            h[i] = static_cast<float>(rand()) / RAND_MAX;

        float* d_in = nullptr;                       // Указатель на GPU-вход
        float* d_out = nullptr;                      // Указатель на GPU-выход

        CHECK_CUDA(cudaMalloc(&d_in, N * sizeof(float))); // Выделение памяти
        CHECK_CUDA(cudaMemcpy(d_in, h.data(), N * sizeof(float), cudaMemcpyHostToDevice)); // Копирование
        CHECK_CUDA(cudaMalloc(&d_out, sizeof(float))); // Память под результат

        auto t0 = chrono::high_resolution_clock::now(); // Начало CPU таймера
        cpu_reduce(h);                               // CPU редукция
        auto t1 = chrono::high_resolution_clock::now(); // Конец таймера
        double cpu_ms = chrono::duration<double, milli>(t1 - t0).count(); // Время CPU

        CHECK_CUDA(cudaMemset(d_out, 0, sizeof(float))); // Обнуление результата
        auto t2 = chrono::high_resolution_clock::now(); // Начало GPU atomic
        reduce_atomic_kernel<<<(N + 255) / 256, 256>>>(d_in, d_out, N); // Запуск atomic ядра
        CHECK_CUDA(cudaDeviceSynchronize());        // Ожидание
        auto t3 = chrono::high_resolution_clock::now(); // Конец таймера
        double gpu_atomic_ms = chrono::duration<double, milli>(t3 - t2).count(); // Время atomic

        auto t4 = chrono::high_resolution_clock::now(); // Начало shared редукции
        gpu_reduce_shared(d_in, N);                 // Оптимизированная GPU редукция
        auto t5 = chrono::high_resolution_clock::now(); // Конец таймера
        double gpu_shared_ms = chrono::duration<double, milli>(t5 - t4).count(); // Время shared

        cout << setw(10) << N                        // Вывод строки таблицы
             << setw(15) << cpu_ms
             << setw(20) << gpu_atomic_ms
             << setw(20) << gpu_shared_ms
             << endl;

        cudaFree(d_in);                              // Освобождение GPU памяти
        cudaFree(d_out);
    }

    return 0;                                       // Завершение программы
}

Overwriting benchmark.cu


In [12]:
!nvcc -arch=sm_75 -O2 benchmark.cu -o benchmark
!./benchmark | head -n 20

         N     CPU (мс)   GPU atomic (мс)   GPU shared (мс)
      1024          0.000               0.157               0.119
      4096          0.000               0.033               0.082
     16384          0.000               0.074               0.085
     65536          0.000               0.256               0.091
    262144          0.000               1.006               0.363
   1048576          0.000               3.711               0.521


##**Вывод**

В ходе выполнения задания было проведено измерение времени выполнения операции редукции для массивов различного размера на CPU и GPU. На стороне GPU были рассмотрены две реализации: наивная версия с использованием атомарных операций в глобальной памяти и оптимизированная версия с применением разделяемой памяти.

Результаты показали, что при малых размерах массива использование GPU не даёт выигрыша по времени по сравнению с CPU, что связано с накладными расходами на запуск CUDA-ядра. При увеличении размера массива наивная GPU-реализация с использованием атомарных операций демонстрирует существенное ухудшение производительности из-за сериализации доступа к глобальной памяти. Для массива размером 1 048 576 элементов время выполнения данной реализации составило около 3.7 мс.

Оптимизированная версия редукции с использованием разделяемой памяти показала значительно лучшую масштабируемость. Основная часть вычислений выполняется внутри блоков в shared memory, что снижает количество обращений к глобальной памяти. Для массива размером 1 048 576 элементов время выполнения составило около 0.52 мс, что обеспечивает ускорение более чем в 7 раз по сравнению с атомарной реализацией.

Таким образом, результаты эксперимента подтверждают эффективность использования разделяемой памяти для оптимизации операций редукции на GPU и демонстрируют преимущества оптимизированных параллельных алгоритмов при работе с большими объёмами данных.